In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import os
import torch
import itertools
import glob
import pandas as pd
from pathlib import Path
import mubind as mb
from tqdm.notebook import tqdm_notebook as tqdm

Fontconfig warning: ignoring UTF-8: not a valid region tag


In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device: " + str(device))

# suppress numba deprecations warnings
from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
import warnings

warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaPendingDeprecationWarning)

Using device: cpu


In [5]:
# path to dir containing pwm txt files
base_path = Path('/Users/egeerdogan/Downloads/Homo_sapiens/pwms_all_motifs')

Load all pwms into separate dataframes.

In [6]:
# collect paths to all pwms
motif_paths = []
for p in base_path.rglob('*'):
    motif_paths.append(p)

pwms = [pd.read_csv(p, sep='\t').drop(columns=['Pos'], axis=1) for p in motif_paths]

In [7]:
pwms[0]

,A,C,G,T
0,0.126205,0.449849,0.241867,0.182078
1,0.115512,0.455572,0.265060,0.163855
2,0.116114,0.515813,0.167470,0.200602
3,0.159639,0.449849,0.132982,0.257530
4,0.205271,0.315512,0.367169,0.112048
5,0.092771,0.239157,0.553916,0.114157
6,0.037048,0.734639,0.133133,0.095181
7,0.056325,0.719277,0.034940,0.189458
8,0.106325,0.871687,0.014759,0.007229
9,0.031024,0.890211,0.034337,0.044428


**TODO**: assign a `Multibind` instance to `multibind_model`.

In [ ]:
# TODO 
multibind_model = None 

filter_dists = [] # list of distance lists for each multibind filter
for mb_filter in multibind_model.binding_modes.conv_mono[1:]: # skip first None filter
    dists = [] # distance list of one multibind filter for each pwm
    for i, pwm in enumerate(tqdm(pwms, leave=False)):
        if pwm.shape[0] > 5: 
            a = mb_filter.weight.detach().numpy()
            a = a.reshape((4, a.shape[-1]))
            b = pwm.values.T 
            d = mb.tl.min_distance(a, b)
            dists.append(d)
    filter_dists.append(dists)

In [ ]:
print('Layer\tMin dist\t\tIndex')
for i, dists in enumerate(filter_dists):
    min_dist = min(dists)
    print(f'{i}\t{min_dist}\t{dists.index(min_dist)}')